In [1]:
# File for regression

In [2]:
from os.path import join
from google.colab import drive

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'My Drive/Capstone Project'
ROOT = '/content/gdrive/'
drive.mount(ROOT)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

In [4]:
#Setting Random Seed
np.random.seed(0)
torch.manual_seed(0)

In [5]:
#Setting Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
data_dir = ROOT + MY_GOOGLE_DRIVE_PATH + '/Edge/data/'

In [7]:
mv = pd.read_table(data_dir + 'mv.dat',skiprows=15,delimiter=",",header=None)
mv.columns = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 
                     'X7', 'X8', 'X9', 'X10', 'Y']

california = pd.read_table(data_dir + 'california.dat',skiprows=13,delimiter=",",header=None)
california.columns = ['Longitude', 'Latitude', 'HousingMedianAge', 'TotalRooms', 'TotalBedrooms', 'Population', 
                     'Households', 'MedianIncome', 'MedianHouseValue']

print('Shape - MV: {} & California: {}'.format(mv.shape, california.shape))

Shape - MV: (40768, 11) & California: (20640, 9)


In [8]:
mv.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y
0,-4.809150,-10.5062,0,-5.25312,0.665394,-23.62270,0,1,445.012,1082,-2.40458
1,-0.928705,-10.8176,1,-5.40878,0.430141,-25.32900,0,0,250.038,1115,-26.25770
2,1.304560,-11.9609,0,-5.98045,0.815231,-7.46146,0,1,261.694,1163,0.65228
3,-4.068760,-11.7433,0,-5.87165,0.285156,-2.89871,1,0,175.301,1191,-6.96747
4,4.750250,-12.9444,2,-6.47220,0.701619,-12.84180,1,1,341.723,1010,2.37512


In [9]:
mv_scaler = StandardScaler()

In [10]:
data = mv_scaler.fit_transform(mv.iloc[:, :-1])
target = np.asarray(mv.Y)

data = data.astype(np.float32)
target = target.astype(np.float32)

mv = {'data': torch.from_numpy(data), 'target': torch.from_numpy(target).unsqueeze(1)}

In [11]:
mv_dataset = torch.utils.data.TensorDataset(mv["data"],mv["target"])

In [12]:
#Model Architecture
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0]) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])  
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size[1], num_classes)  

    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)

        return out

In [13]:
#hyperparameters
input_size = mv["data"].shape[1]
hidden_size = [10,20]
num_classes = 1
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [14]:
train_loader = torch.utils.data.DataLoader(dataset=mv_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

In [15]:
model_mv = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_mv.parameters(), lr=learning_rate)  

In [16]:
print(model_mv)

NeuralNet(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=10, out_features=20, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=20, out_features=1, bias=True)
)


In [17]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_mv(features)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/408], Loss: 178.7869
Epoch [1/5], Step [200/408], Loss: 86.6957
Epoch [1/5], Step [300/408], Loss: 17.2859
Epoch [1/5], Step [400/408], Loss: 14.1963
Epoch [2/5], Step [100/408], Loss: 4.8581
Epoch [2/5], Step [200/408], Loss: 3.1737
Epoch [2/5], Step [300/408], Loss: 1.5900
Epoch [2/5], Step [400/408], Loss: 0.8026
Epoch [3/5], Step [100/408], Loss: 0.7530
Epoch [3/5], Step [200/408], Loss: 0.6830
Epoch [3/5], Step [300/408], Loss: 0.5028
Epoch [3/5], Step [400/408], Loss: 0.6351
Epoch [4/5], Step [100/408], Loss: 0.3674
Epoch [4/5], Step [200/408], Loss: 0.3612
Epoch [4/5], Step [300/408], Loss: 0.1842
Epoch [4/5], Step [400/408], Loss: 0.2152
Epoch [5/5], Step [100/408], Loss: 0.1733
Epoch [5/5], Step [200/408], Loss: 0.1383
Epoch [5/5], Step [300/408], Loss: 0.1863
Epoch [5/5], Step [400/408], Loss: 0.0974


In [18]:
l = [module for module in model_mv.children() if type(module) != nn.Sequential]

In [21]:
for i in l:
  try:
    print('Layer: {} weight:{}'.format(i,i.weight))
  except:
    continue

Layer: Linear(in_features=10, out_features=10, bias=True) weight:Parameter containing:
tensor([[ 0.3536,  0.0793,  0.0288, -0.4291, -0.1315, -0.3280, -0.0195, -0.1729,
          0.0263,  0.0102],
        [-0.4497, -0.1353, -0.2951, -0.3943, -0.2765, -0.4555,  0.1074, -0.3614,
         -0.2262, -0.1972],
        [-0.1457,  0.0529, -0.0205, -0.0367, -0.0551, -0.6343,  0.0986, -0.9802,
         -0.1138, -0.0478],
        [-0.3958,  0.0817, -0.1935, -0.3617, -0.4223, -0.7626, -0.3647, -0.3351,
          0.2105,  0.1243],
        [-0.0816, -0.1775,  0.2291, -0.4368, -0.3200, -0.5963,  0.1431, -0.4103,
         -0.1139, -0.0091],
        [-0.0326, -0.0270, -0.0054,  0.0510,  0.1108, -0.7117, -0.0167, -1.0686,
          0.0067, -0.0021],
        [-0.1043,  0.0674, -0.1296, -0.1801, -0.0483, -0.6387,  0.0269, -0.7602,
          0.1561,  0.3007],
        [ 0.5304,  0.0110,  0.0112, -0.0653, -0.1917,  0.7905,  0.0407, -0.1045,
         -0.0778, -0.0574],
        [-0.4298,  0.1083,  0.2513, -0.45

In [20]:
print(l[0].weight.cpu().detach().numpy())

[[ 0.35357723  0.07926894  0.02878217 -0.42913762 -0.1314955  -0.32799163
  -0.01954031 -0.17289057  0.0263077   0.01020833]
 [-0.44973883 -0.13530812 -0.29510412 -0.39426044 -0.2764542  -0.45545024
   0.1073738  -0.3614438  -0.22624974 -0.19717905]
 [-0.1457034   0.05287194 -0.02054278 -0.03671077 -0.05507015 -0.634343
   0.09861298 -0.9802053  -0.11382522 -0.04783566]
 [-0.39576825  0.08166213 -0.19352886 -0.36172608 -0.42225584 -0.7626294
  -0.364678   -0.33506656  0.21052204  0.12430433]
 [-0.08157441 -0.17748305  0.22907941 -0.4367811  -0.31995228 -0.596347
   0.14309017 -0.41027236 -0.11389344 -0.00914161]
 [-0.03258049 -0.02700445 -0.0054154   0.05100203  0.11084107 -0.7117338
  -0.0166628  -1.0685872   0.00673723 -0.00205655]
 [-0.10432629  0.06737386 -0.12960258 -0.18008359 -0.04825691 -0.63869745
   0.02686094 -0.760165    0.15610148  0.30065346]
 [ 0.5304321   0.01099884  0.01116842 -0.06529903 -0.19167066  0.7904549
   0.04070542 -0.10451909 -0.0778056  -0.05743451]
 [-0.42